## Descriptions:
- Check the characteristics of the cohort with predicted probability difference = 0.3) and transfer from 0 to 1

**Inputs**:  
- `1_4_cohort_test_results`
- `1_4_cohort_all_current_dx`
    
**Outputs**: (did not save anything here)
- `4_3_diff23tx01IDs` cohort test of abs_diff from 0.2 to 0.3 and transferred from 0 to 1 --> to re-id

### Importing R libraries

In [1]:
library(bigrquery)  # to query STARR-OMOP (stored in BigQuery) using SQL
library(tidyverse)
library(lubridate)

# options(repr.matrix.max.rows=250, repr.matrix.max.cols=30)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [2]:
# directories
datadir = "../../DataTD"
cohortdir = "../../OutputTD/1_cohort"
featuredir = "../../OutputTD/2_features"
modeldir4 = "../../OutputTD/3_models/1_4_cohort"
# modeldir4preadmit = "../../OutputTD/3_models/1_4_cohort_24hrpreadmit"
tabledir = "../../OutputTD/4_tables"

options(repr.matrix.max.rows=200, repr.matrix.max.cols=30)

In [3]:
cohort <- read.csv(file.path(modeldir4, "1_4_cohort_test_results.csv")) %>% distinct() %>% 
            select(anon_id, pat_enc_csn_id_coded, admit_time, 
                   first_label, death_24hr_recent_label, death_24hr_max_label,
                   pred_first, pred_death_24hr_recent, transfer, 
                   abs_diff0_24, diff0_True, diff24_True) 
nrow(cohort) # 2604
nrow(cohort %>% distinct())
nrow(cohort %>% select(anon_id) %>% distinct()) # 2328
nrow(cohort %>% select(pat_enc_csn_id_coded) %>% distinct()) # 2604

# count anon_id and csn, count unique in each group, but possible to have same ids in different groups
cohort %>%  distinct() %>% group_by(abs_diff0_24) %>% 
            summarise(nrows = n(), count_csn = n_distinct(pat_enc_csn_id_coded), count_mrn = n_distinct(anon_id)) %>% 
            arrange(-abs_diff0_24) %>% mutate(cum_csn = cumsum(count_csn), cum_mrn = cumsum(count_mrn))

[1] 12418

[1] 12418

[1] 9780

[1] 12418

abs_diff0_24,nrows,count_csn,count_mrn,cum_csn,cum_mrn
<dbl>,<int>,<int>,<int>,<int>,<int>
0.7,3,3,3,3,3
0.6,7,7,7,10,10
0.5,27,27,24,37,34
0.4,80,80,77,117,111
0.3,206,206,204,323,315
0.2,545,545,525,868,840
0.1,1736,1736,1620,2604,2460
0.0,9814,9814,7979,12418,10439


In [7]:
# get cohort with abs_diff from 0.2 to 0.3 and with transfer
sum(cohort$transfer)

nrow(cohort %>% filter(abs_diff0_24 >=0.2, abs_diff0_24 <=0.3, transfer==1) %>% distinct(pat_enc_csn_id_coded))


nrow(cohort %>% filter(abs_diff0_24 >=0.2, abs_diff0_24 <=0.3, first_label == 1, death_24hr_recent_label ==0) %>% 
                distinct(pat_enc_csn_id_coded)) # 139

nrow(cohort %>% filter(abs_diff0_24 >=0.2, abs_diff0_24 <=0.3, first_label == 0, death_24hr_recent_label ==1) %>% 
                distinct(pat_enc_csn_id_coded)) #36

diff23tx01 <- cohort %>% filter(abs_diff0_24 >=0.2, abs_diff0_24 <=0.3, first_label == 0, death_24hr_recent_label ==1) %>% 
                distinct(anon_id) # 89 csn for absdiff from 0.1 to 0.3 and 36 csn for 0.2 to 0.3

nrow(diff23tx01)
# write.csv(diff23tx01, file.path(tabledir, "4_3_diff23tx01IDs_35.csv"), row.names=FALSE)

[1] 805

[1] 175

[1] 139

[1] 36

[1] 35

### Checking diagnosis to compare with total cohort

In [5]:
# all dx from 1_4_cohort, SQL query
dx <- read.csv(file.path(datadir, "1_4_cohort_all_current_dx.csv"))
nrow(dx) #1239918

ndx = length(unique(dx$pat_enc_csn_id_coded)) # full cohort is 43980
ndx

[1] 1239918

[1] 43932

In [6]:
# cohort transfers with diagnoses
dx_diff <- left_join(diff23tx01, dx)
nrow(dx_diff)
ndf = length(unique(dx_diff$pat_enc_csn_id_coded))
ndf
length(unique(dx_diff$anon_id))

dx_diff %>% distinct(pat_enc_csn_id_coded, dx_name) %>% count(dx_name, sort=TRUE) %>% head(20)

Joining, by = "anon_id"



[1] 3088

[1] 89

[1] 35

,dx_name,n
,<chr>,<int>
1,Other long term (current) drug therapy,37
2,Type 2 diabetes mellitus with diabetic chronic kidney disease,28
3,"Acute kidney failure, unspecified",26
4,"Hyperlipidemia, unspecified",25
5,Acidosis,24
6,Personal history of nicotine dependence,24
7,Long term (current) use of insulin,23
8,Weakness,22
9,Atherosclerotic heart disease of native coronary artery without angina pectoris,21


In [41]:
# only 117 of obs above 0.4
# all visits in diff4 have diagnoses

ndfn = nrow(dx_diff %>% distinct(pat_enc_csn_id_coded, dx_name))
ndfn

length(setdiff(dx_diff$pat_enc_csn_id_coded, dx$pat_enc_csn_id_coded))
length(setdiff(dx$pat_enc_csn_id_coded, dx_diff$pat_enc_csn_id_coded))

colnames(dx_diff)

[1] 2987

[1] 0

[1] 43843

[1] "anon_id"              "pat_enc_csn_id_coded" "line"                
 [4] "dx_name"              "primary"              "chronic"             
 [7] "principal"            "hospital_pl"          "ed"                  
[10] "present_on_adm"

In [8]:
summary(dx_diff$line)

dx_diff %>% count(primary, sort=TRUE)
dx_diff %>% count(chronic, sort=TRUE)
dx_diff %>% count(hospital_pl,  sort=TRUE)
dx_diff %>% count(principal, sort=TRUE)
dx_diff %>% count(ed, sort=TRUE)
dx_diff %>% count(present_on_adm, sort=TRUE)
# dx_diff %>% count(dx_name, sort=TRUE) %>% head(20)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
    1.0     4.0    10.0    13.1    20.0    70.0      67 

primary,n
<chr>,<int>
,2472
N,514
Y,102


chronic,n
<chr>,<int>
,2436
N,641
Y,11


hospital_pl,n
<chr>,<int>
,3021
Y,50
N,17


principal,n
<chr>,<int>
,3021
N,60
Y,7


ed,n
<chr>,<int>
,2503
N,434
Y,151


present_on_adm,n
<chr>,<int>
Yes,1544
,745
Exempt from POA reporting,505
No,276
Unknown,18


In [9]:
dx_diff %>% filter(ed=="Y", primary=="Y") %>% distinct(pat_enc_csn_id_coded, dx_name) %>% 
            count(dx_name, sort=TRUE) %>% filter(n>1)

dx_name,n
<chr>,<int>
Hypoxia,3
"Acute on chronic heart failure, unspecified heart failure type (CMS-HCC)",2
Cholangiocarcinoma (CMS-HCC),2
HCAP (healthcare-associated pneumonia),2
"Motor vehicle collision, initial encounter",2
Syncope and collapse,2
